In [2]:
# importing libraries
import cartopy.feature as cf
import cartopy.crs as ccrs
import xarray as xr
import matplotlib.pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import geopandas as gpd
import numpy as np
import matplotlib.colors as colors
import matplotlib.colorbar as colorbar
from matplotlib.colors import LinearSegmentedColormap
import os
import glob
from PIL import Image
import gc

In [9]:
def load_data(path):
    try:
        data = xr.open_dataset(path).rename(dict(x='lon', y='lat', t='time'))
        # data.attrs = {'long_name': 'NDVI', 'units': '-'}
        
    except FileNotFoundError:
        data = None
    
    return data

def nan_mask(array, band_name, title):
    nan_mask = np.isnan(array)
    if np.any(nan_mask):
        # print(f"{title} has NaN values in {band_name} Band")
        return True
    else:
        return False

def png_export(datacube, path, extent, percentile = None): 

    if not os.path.exists(path):
        os.makedirs(path)

    for i in range (len(datacube['time'])):
        date = str(datacube['time'].values[i])[:10]
        title = f'Bumba Extent {extent}: {date}'
        filename = f'{path}/ex{extent}_{date}.png'

        # Selecting data at the first time index
        selected_time_data = datacube.isel(time=i)  # Use sel(time='your_time_value') if you know the exact time value

        # Extracting specific bands for the true color image
        blue = selected_time_data['B02']
        green = selected_time_data['B03']
        red = selected_time_data['B04']

        nan_b = nan_mask(blue, 'Blue', title)
        nan_g = nan_mask(green, 'Green', title)
        nan_r = nan_mask(red, 'Red', title)

        if not (nan_b or nan_g or nan_r):

            # Normalization function
            if percentile is None:
                def normalize(band):
                    return (band - band.min()) / (band.max() - band.min())
            
            else:
                def normalize(band, percentile):
                    lower = np.percentile(band, percentile[0])
                    upper = np.percentile(band, percentile[1])
                    normalized = (band - lower) / (upper - lower)
                    return np.clip(normalized, 0, 1)
            
            blue_norm = normalize(blue, percentile)
            green_norm = normalize(green, percentile)
            red_norm = normalize(red, percentile)

            # Combine into an RGB array
            rgb = np.stack([red_norm, green_norm, blue_norm], axis=-1)

            # Set up figure size based on the data aspect ratio
            aspect_ratio = blue.shape[1] / blue.shape[0]  # Assuming width/height
            fig_width = 7 # You can adjust this size to better fit your screen or output medium
            fig_height = fig_width / aspect_ratio

            plt.figure(figsize=(fig_width, fig_height))
            plt.imshow(rgb)
            plt.title(title)  # Add your title here
            plt.axis('off')  # Hide axes to enhance the image focus
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            plt.close()
        
        else:
            print(f"Skipping {title} due to NaN values")

In [68]:
# paths = ["./data/bumba_RGB_ex1.nc", "./data/bumba_RGB_ex2.nc"]

# for i in range(2):
#     datacube = load_data(paths[i])
#     png_export(datacube, './data/output2', i+1)
#     del datacube

In [10]:
paths = ["./data/bumba_RGB_ex1.nc", "./data/bumba_RGB_ex2.nc"]

datacube = load_data(paths[1])
png_export(datacube, './data/extent2/advanced_norm3', 2, percentile = [25, 75])

Skipping Bumba Extent 2: 2020-05-21 due to NaN values
Skipping Bumba Extent 2: 2020-06-25 due to NaN values
Skipping Bumba Extent 2: 2020-06-30 due to NaN values
Skipping Bumba Extent 2: 2020-07-05 due to NaN values
Skipping Bumba Extent 2: 2020-07-30 due to NaN values
Skipping Bumba Extent 2: 2020-08-14 due to NaN values
Skipping Bumba Extent 2: 2020-08-19 due to NaN values
